# Optimal Replacement of GMC Bus Engines: An empirical model of Harold Zurcher

This notebook replicates the descriptives in Tabla IIa and IIb from
> Rust, J. (1987). [Optimal Replacement of GMC Bus Engines: An empirical model of Harold Zurcher.](https://doi.org/10.2307/1911259) *Econometrica*, Vol. 55, No.5, 999-1033. 

The data is taken from the NFXP software provided by [Rust](https://editorialexpress.com/jrust/nfxp.html) which is available to download [here](https://github.com/OpenSourceEconomics/ruspy/tree/master/data). 

## Preparations

Before executing this file the raw data needs to be processed.

In [1]:
from data.data_reading import data_reading
from data.data_location import get_data_storage

In [2]:
data_reading()

### Odometer at Engine Replacement
Table IIa in Rust's paper describes the milage on which a engine replacement occured. As there are buses, which had a second replacement during the time of the observation, the record of the second replacement will be reduced by the milage of the first, to get the real life time milage of an engine.

In [3]:
import pandas as pd
import os
data_path = get_data_storage()
dict_df = dict()
for filename in os.listdir(data_path + '/pkl/group_data/'):
    if filename.endswith(".pkl"):
        dict_df[filename[0:7]] = pd.read_pickle(data_path + '/pkl/group_data/' + filename)

In [7]:
df = pd.DataFrame()
for j, i in enumerate(sorted(dict_df.keys())):
    df2 = dict_df[i][['Odo_1']][dict_df[i]['Odo_1'] > 0]
    df2 = df2.rename(columns={'Odo_1': i})
    df3 = dict_df[i][['Odo_2']].sub(dict_df[i]['Odo_1'], axis=0)[dict_df[i]['Odo_2'] > 0]
    df3 = df3.rename(columns={'Odo_2': i})
    df3 = df3.set_index(df3.index.astype(str) + '_2')
    df4 = pd.concat([df2, df3])
    if j == 0:
        df = df4.describe()
    else:
        df = pd.concat([df, df4.describe()], axis=1)
df = df.transpose()
df = df.drop(df.columns[[4, 5, 6]], axis=1)
df[['max', 'min', 'mean', 'std', 'count']].fillna(0).astype(int)

max     min    mean    std  count
group_1       0       0       0      0      0
group_2       0       0       0      0      0
group_3  273400  124800  199733  37459     27
group_4  387300  121300  257336  65477     33
group_5  322500  118000  245290  60257     11
group_6  237200   82400  150785  61006      7
group_7  331800  121000  208962  48980     27
group_8  297500  132000  186700  43956     19

### Never failing buses

Table IIb explores buses, which never had an engine replacement. Therefore this data is left-censored, as the econometrican never observes the time of replacement. The table shows the variation in the odometer record at the end of the observation period.

In [8]:
df = pd.DataFrame()
for i in sorted(dict_df.keys()):
    df2 = dict_df[i][[dict_df[i].columns.values[-1]]][dict_df[i]['Odo_1'] == 0]
    df2 = df2.rename(columns={df2.columns.values[0]: i})
    if j == 0:
        df = df2.describe()
    else:
        df = pd.concat([df, df2.describe()], axis=1)
df = df.transpose()
df = df.drop(df.columns[[4, 5, 6]], axis=1)
df[['max', 'min', 'mean', 'std', 'count']].fillna(0).astype(int)

max     min    mean    std  count
group_1  120151   65643  100116  12929     15
group_2  161748  142009  151182   8529      4
group_3  280802  199626  250766  21324     21
group_4  352450  310910  337221  17802      5
group_5  326843  326843  326843      0      1
group_6  299040  232395  265263  33331      3
group_7       0       0       0      0      0
group_8       0       0       0      0      0